In [77]:
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
import rdata
import numpy as np

In [78]:
nacc_test = pd.read_csv(r'C:\Users\jaiga\Downloads\UDS3Mod_for_Jai_Crosswalk_Levels_only.csv')
nacc_test.head(3)

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_data_access_group,redcap_survey_identifier,adc_sub_id,visitmo,visitday,visityr,...,pbmc_count,pbmc_dt,plasma_count,plasma_dt,serum_count,serum_dt,urine_count,urine_dt,biospe_dde_notes,biospecimen_inventory_complete
0,104,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1105,1,14,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1102,1,13,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,106,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1129,2,5,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
nacc_test.shape

(3, 2900)

In [80]:
uds3_df = nacc_test
uds3_df.columns = uds3_df.columns.str.lower()

In [81]:
uds3_df.shape

(3, 2900)

In [82]:
uds3_df['telmod'].value_counts()

telmod
1.0    1
Name: count, dtype: int64

In [83]:
# Identify all float columns
float_columns = uds3_df.select_dtypes(include=['float']).columns

# Convert safely
for col in float_columns:
    # Convert to numeric, coerce errors to NaN
    uds3_df[col] = pd.to_numeric(uds3_df[col], errors='coerce')

    # Replace infinities with NaN
    uds3_df[col] = uds3_df[col].replace([np.inf, -np.inf], np.nan)

    # Check if all remaining values are safe for integer conversion
    if uds3_df[col].dropna().apply(float.is_integer).all():
        uds3_df[col] = uds3_df[col].astype("Int64")  # Convert to nullable integer
    else:
        print(f"Skipping column '{col}' due to non-integer values.")

# Check the updated data types
print(uds3_df.dtypes)


Skipping column 'form_ver_num' due to non-integer values.
Skipping column 'height' due to non-integer values.
Skipping column 'bmi' due to non-integer values.
Skipping column 'inbody_skel_musc_mass' due to non-integer values.
Skipping column 'inbody_body_fat_mass' due to non-integer values.
Skipping column 'inbody_bmi' due to non-integer values.
Skipping column 'inbody_perc_body_fat' due to non-integer values.
Skipping column 'memory' due to non-integer values.
Skipping column 'orient' due to non-integer values.
Skipping column 'commun' due to non-integer values.
Skipping column 'homehobb' due to non-integer values.
Skipping column 'cdrsum' due to non-integer values.
Skipping column 'tot_sum' due to non-integer values.
Skipping column 'cdrglob' due to non-integer values.
Skipping column 'comport' due to non-integer values.
Skipping column 'orient_dist' due to non-integer values.
Skipping column 'judgment_dist' due to non-integer values.
Skipping column 'commun_dist' due to non-integer 

In [84]:
uds4_df = pd.DataFrame()

# Load JSON mapping file
with open(r'C:\Users\jaiga\Downloads\Crosswalks\A5_D2_UDS4_programmatic_crosswalk_mappings.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)
    
# Define the categories and their respective suffixes
categories = {
    'a': [1, 2, 3, 4,5],
    'b': [1, 4, 5, 6, 7, 8, 9],
    'c': [2],
    'd': [1,2]
}

# Initialize dictionary
data_dict = {}
for category, numbers in categories.items():
    for num in numbers:
        key_suffix = f"__{category}{num}"
        data_dict[f'ptid{key_suffix}'] = 'adc_sub_id'
        data_dict[f'visitnum{key_suffix}'] = f'{category}{num}_visit_day'
        data_dict[f'initials{key_suffix}'] = f'{category}{num}_ex_ini'

In [103]:
uds4_df = pd.DataFrame()

def process_mappings(mapping_type):
    for mapping in mapping_data.get(mapping_type, []):
        uds3_var = mapping["UDS3_variable"].lower()
        uds4_var = mapping["UDS4_variable"].lower()

        uds3_var = data_dict.get(uds3_var, uds3_var)
        structured_mappings = []

        if mapping_type == 'Direct_Mappings':
            response_levels = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Conformity"]
        else:
            response_levels = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Conformity"]
            structured_mappings = [m for m in mapping.get("crosswalk_mappings", []) if m["mapping_type"] == "Structured mapping"]
        
        if mapping_type == 'Structured_Transformations':
            if response_levels and not conformity and uds3_var in uds3_df.columns:
                response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}
                for uds3_value, uds4_value in response_map.items():
                    if "|" not in uds3_value and "grep(" not in uds3_value:
                        if uds4_var in uds4_df.columns:
                            mm = uds4_df[uds4_var] == "NA"
                            uds4_df.loc[mm & uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value
                        else:
                            uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value

        if structured_mappings:
            for struct_map in structured_mappings:
                for mapping_entry in struct_map["mappings"]:
                    uds3_value = mapping_entry["UDS3_value"]
                    uds4_value = mapping_entry["UDS4_value"]

                    # Identify separator: OR ('|') or AND ('&')
                    if "|" in uds3_value:
                        separator = "|"
                        operator = np.logical_or
                    elif "&" in uds3_value:
                        separator = "&"
                        operator = np.logical_and
                    else:
                        separator = None
                        operator = None

                    # Handle multiple UDS3 variables with OR / AND conditions
                    if separator:
                        uds3_vars = [var.strip() for var in uds3_var.split(f" {separator} ")]
                        uds3_values = [var.strip() for var in uds3_value.split(f" {separator} ")]

                        # Ensure all UDS3 columns exist in the DataFrame
                        if all(col in uds3_df.columns for col in uds3_vars):
                            conditions = [(uds3_df[col].astype(str) == val) for col, val in zip(uds3_vars, uds3_values)]
                            # Apply OR or AND depending on the separator
                            if separator == '|':
                                mask = np.logical_or.reduce(conditions)
                            else:
                                mask = np.logical_and.reduce(conditions)

                            # Only update where conditions match
                            if uds4_var in uds4_df.columns:
                                missing_mask = uds4_df[uds4_var].isna()  # Handle missing (NA) values in uds4_df
                                uds4_df.loc[missing_mask & mask, uds4_var] = uds4_value
                            else:
                                uds4_df.loc[mask, uds4_var] = uds4_value
                    else:
                        
                        # Handle structured mathematical mappings (e.g., "PDAGE + BIRTHYR" → "PDYR - BIRTHYR")
                        if any(op in uds3_value for op in ["+", "-", "*", "/"]):
                            uds3_cols = [col.lower() for col in re.findall(r'[A-Za-z0-9_]+', uds4_value)]

                            # Ensure all columns exist before computing
                            if all(col in uds3_df.columns for col in uds3_cols):
                                uds3_expr = uds4_value.replace("–", "-").strip().lower()
                                for col in uds3_cols:
                                    uds3_expr = uds3_expr.replace(col, f"uds3_df['{col}'].fillna(0)")

                            try:
                                uds3_computed = eval(uds3_expr, {"uds3_df": uds3_df, "np": np})
                            except Exception as e:
                                print(f"Error evaluating UDS3 expression {uds3_expr}: {e}")
                                continue
                            if uds4_var and uds4_var not in uds4_df.columns:
                                uds4_df[uds4_var] = "NA"

                            if uds4_var:
                                uds4_df[uds4_var] = uds3_computed.astype(int)

                    # Handle "paste()" format for combining multiple columns into a single string
                    if "paste(" in uds4_value:
                        components_raw = uds4_value.replace("paste(", "").replace(")", "").split(',')
                        columns = [
                            re.sub(r'(_{1,2}[A-E]\d*)$', '', comp.strip()).lower()
                            for comp in components_raw
                            if re.search(r'_{1,2}[A-E]', comp)
                        ]  # Checking if it contains a valid suffix like _A, __A, _B, __B, etc.

                        if all(comp in uds3_df.columns for comp in columns):
                            uds4_df[uds4_var] = (
                                uds3_df[columns[0]].astype(str) + "/" +
                                uds3_df[columns[1]].astype(str) + "/" +
                                uds3_df[columns[2]].astype(str)
                            )

       # Case 1: No Response LEVELS, copy values directly
        if not response_levels:
            if uds3_var in uds3_df.columns:
                if uds4_var in uds4_df.columns:
                    mm = uds4_df[uds4_var].isna()
                    uds4_df.loc[mm, uds4_var] = uds3_df[uds3_var]
                else:
                    uds4_df[uds4_var] = uds3_df[uds3_var]

        # Case 2: Conformity check
        elif conformity:
            uds3_conformity_values = set(item["UDS3_value"] for mapping in conformity for item in mapping["mappings"])
            uds4_conformity_values = set(item["UDS4_value"] for mapping in conformity for item in mapping["mappings"])
            
            if uds3_conformity_values == uds4_conformity_values:  # Ensure conformity matches
                if uds3_var in uds3_df.columns:
                    uds4_df[uds4_var] = uds3_df[uds3_var]
            else:
                response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}
                    
                if uds3_var in uds3_df.columns:
                    # Applying exact single-value mappings first (e.g., 2 -> 1)
                    for uds3_value, uds4_value in response_map.items():
                        if "|" not in uds3_value and "grep(" not in uds3_value:
                            if pd.api.types.is_float_dtype(uds3_df[uds3_var]):
                                float_uds3_value = float(uds3_value)
                                uds4_df.loc[uds3_df[uds3_var] == float_uds3_value, uds4_var] = uds4_value

                            else:
                                uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value
                        else:
                            if not (uds3_var.endswith("sec") or uds3_var.endswith("ter")): #ignoring rest of racesec since it wont add any value
                                # Handling cases with logical OR conditions (e.g., "1 | 3 | 4 | 5 | 50 | 99" -> None)
                                values = uds3_value.split(" | ")
                                uds4_df.loc[uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value
                    
                    # Handling grep-based text searches (e.g., grep("guatemalan", HISPORX))
                    for uds3_value, uds4_value in response_map.items():
                        if "grep(" in uds3_value:
                            search_term = re.search(r'grep\("(.*?)",', uds3_value).group(1).lower()
                            uds4_df.loc[uds3_df[uds3_var].astype(str).str.contains(search_term, case=False, na=False), uds4_var] = uds4_value
        
        # Case 3: Handling secondary and tertiary race variables (race, racesec, raceter)
        if uds3_var.endswith("sec") or uds3_var.endswith("ter"):
            if uds3_var in uds3_df.columns:
                missing_mask = uds4_df[uds4_var] == "NA"  # Identify rows where primary race is NA
                
                # Only update NA values in UDS4 based on racesec or raceter mappings
                for uds3_value, uds4_value in response_map.items():
                    if "|" not in uds3_value and "grep(" not in uds3_value:
                        uds4_df.loc[missing_mask & (uds3_df[uds3_var].astype(str) == uds3_value), uds4_var] = uds4_value
                    elif "grep(" in uds3_value:
                        search_term = re.search(r'grep\("(.*?)",', uds3_value).group(1).lower()
                        uds4_df.loc[missing_mask & uds3_df[uds3_var].astype(str).str.contains(search_term, case=False, na=False), uds4_var] = uds4_value
                    else:
                        values = uds3_value.split(" | ")
                        uds4_df.loc[missing_mask & uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value
                        
# Process different mapping types dynamically
mapping_types = ["Direct_Mappings","Conditional_Consistency", "Structured_Transformations","High_Complexity"]  # Add more types as needed
for mapping_type in mapping_types:
    process_mappings(mapping_type)

C:\Users\jaiga\AppData\Local\Temp\ipykernel_17820\1172636212.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '999' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value
C:\Users\jaiga\AppData\Local\Temp\ipykernel_17820\1172636212.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '999' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value
C:\Users\jaiga\AppData\Local\Temp\ipykernel_17820\1172636212.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '999' has dtype incompatible with int32, please explicitly 

In [104]:
# UTF-8-SIG encoding to avoid character corruption
uds4_df.to_csv("uds4_data_a5d2.csv", index=False, na_rep="NA", encoding='utf-8-sig')

In [105]:
uds4_df.head(5)

,cvothrx,othcondx,psycdisx,alcfreqyr,angiocp,anxiety,bipolar,cancer,cancermeta,cancerprim,...,rbd,strokage,lasttbi,rmreasa5d2,modea5d2,tiaage,frmdatea5d2,substpast,substyear,rmmodea5d2
0,<NA>,NaN,<NA>,0,0,0,0,0,NaN,NaN,...,NaN,-1967,-1967,NaN,NaN,-1967,1/14/2025,0,0,NaN
1,<NA>,pectus excavatum requiring Ravitch procedure,<NA>,2,0,0,0,2,NaN,1,...,NaN,-1951,-1951,NaN,NaN,-1951,1/13/2025,0,0,NaN
2,<NA>,NaN,<NA>,3,0,0,1,2,1,NaN,...,NaN,-1960,-1960,NaN,2,999,2/5/2025,0,0,1


In [106]:
uds4_df.shape

(3, 57)

In [107]:
uds4_df.alcfreqyr

0    0
1    2
2    3
Name: alcfreqyr, dtype: object

In [108]:
uds4_df.pdage

0    -1967
1       49
2    -1960
Name: pdage, dtype: object

In [109]:
uds4_df.tiaage

0    -1967
1    -1951
2      999
Name: tiaage, dtype: object

In [110]:
uds4_df.apnea

0    NaN
1    NaN
2    NaN
Name: apnea, dtype: object